## Cropped Cam Sender

Grab video from a webcam, crop the frames, render and send. 


In [ ]:
#!pip install m3u8
#!pip install opencv-python

In [ ]:
import string
import os
import sys
import json
import base64
import ssl
import time
import datetime
import io
import logging
import re
import collections
import threading

import cv2
import requests
import urllib
import m3u8
from urllib.parse import urlparse
from PIL import Image
import numpy as np

import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox, Layout
from IPython.display import clear_output

from interactivecrop.interactivecrop import main as crop

if '../scripts' not in sys.path:
    sys.path.insert(0, '../scripts')

import stream_video as video
import credential

## Tutorial 
A very good walk through on the .m3u8 files that index .ts files that have 
the vido chunks that we need 

https://www.youtube.com/watch?v=bytnxnZFLeg


## test file access
The .ts files have the images to analyize. You need to walk through the index to get to the data. Mulitple index formats. Not sure if thier standard method for the schema, may make it too easy to 'borrow' someone ele's content.


"blob:https://www.earthcam.com/00631398-95e6-40df-9d75-71501c1a6da6"

<video muted="muted" crossorigin="anonymous" id="videoPlayer_html5_api" class="vjs-tech" preload="none" style="position: absolute" tabindex="-1" src="blob:https://www.earthcam.com/00631398-95e6-40df-9d75-71501c1a6da6"></video>



# Locating chunks of video frames.



The live webcams have a hierachy of files that terminates with files with chunks of frames, .ts files. 
The deepest .m3u8 will be updated with every new chunk's arrival. Work your way down to the lowests m3u8 file handle, this file is newly captured .ts files, it gets rewritten with new video chunks arrive. 


## Three examples

Below is collection of sites that we can fetch video via .m3u8 that reference .ts files. 

### whatsupcams 

The standard process is to find a webcam that you are interested in getting the contents from, using 
Chome to look a the network traffic looking for files being fetched that end inf '.m3u8', repeatily. 
Look at the preview, you have the correct file if you see content refering to .ts files like this...
```text
#EXTM3U
#EXT-X-VERSION:3
#EXT-X-MEDIA-SEQUENCE:4982
#EXT-X-TARGETDURATION:12
#EXTINF:11.989,
hr_zadar4-4982.ts
#EXTINF:11.986,
hr_zadar4-4983.ts
#EXTINF:11.989,
hr_zadar4-4984.ts
```


Capture the url link to the file in chrome and past into M3U8 variable in below. 
https://cdn-004.whatsupcams.com/hls/hr_zadar4.m3u8

Below you will find the video link and M3U8 for webcam at different sites


| url | M3U8 |
|:-----|:-----|
|https://www.whatsupcams.com/en/webcams/croatia/zadar/zadar/zadar-narodni-trg-peoples-square/| https://cdn-004.whatsupcams.com/hls/hr_zadar4.m3u8|
| https://www.whatsupcams.com/en/webcams/italy/veneto/venice/webcam-venice-rialto-bridge/ | https://cdn-001.whatsupcams.com/hls/it_venezia01.m3u8| 
| https://www.whatsupcams.com/en/webcams/croatia/zadar/zadar/zadar-trg-petra-zoranica/ | https://cdn-001.whatsupcams.com/hls/hr_zadar3.m3u8 |

        

### caltrans

| url | M3U8 |
|:-----|:-----|
|http://cwwp2.dot.ca.gov/vm/loc/d4/tv388sr1justsouthofpresidiotunnel.htm| http://wzmedia.dot.ca.gov/D4/N1_at_Presidio_Tunnel.stream/chunklist_w665039362.m3u8 |
| http://cwwp2.dot.ca.gov/vm/loc/d4/tv457sr92elcaminoreal.htm | http://wzmedia.dot.ca.gov/D4/W92_at_El_Camino_Real.stream/chunklist_w2118793021.m3u8 |
|http://cwwp2.dot.ca.gov/vm/loc/d4/tve83us101i580.htm | http://wzmedia.dot.ca.gov/D4/N101_at_580.stream/chunklist_w987073694.m3u8 |
| http://cwwp2.dot.ca.gov/vm/loc/d4/tv302us10117thstreet.htm | http://wzmedia.dot.ca.gov/D4/S101_at_17th_St.stream/chunklist_w1949748683.m3u8 |


- Go to the map page, http://cwwp2.dot.ca.gov/vm/iframemap.htm
- select a video camera from the map
- select 'Open this camera in a bookmark-friendly format"
- Using Chrome, grap URL of opened page, for example (http://cwwp2.dot.ca.gov/vm/loc/d4/tv388sr1justsouthofpresidiotunnel.htm). Access the developer tool (View | Developer | Developer Tools) the  'Network' tab
 - Find a 'chunklist*.m3u8' file, the contents of which look like.
 ```
#EXTM3U
#EXT-X-VERSION:4
#EXT-X-TARGETDURATION:11
#EXT-X-MEDIA-SEQUENCE:7817
#EXTINF:9.61,
media_w1296425575_7817.ts
#EXTINF:10.677,
media_w1296425575_7818.ts
#EXTINF:9.61,
media_w1296425575_7819.ts
```
Assign this value to **M3U8** below


Notes : 
- I have found that the caltrans camera are sporatic.

## Huntington beach 
This site has multiple cameras on one page, bring up the page in Chrome's "Network" developer tab. Find the 'chunklist_\*.m3u8' file and set it to the **M3U8** file.

| url | M3U8 |
|:-----|:-----|
| https://hbcams.com/ | https://56cdb389b57ec.streamlock.net:1935/live/hbnorth.stream/chunklist_w790486524.m3u8 |

# Define cropping region for live video streams

The rest of the notebook walks through 
- collecting some sample frames
- selecting frame for cropping 
- cropping the frame
- reviewing the specified cropped region 
- finally streaming the video and rendering the frames cropped.



### Assign the url of the chunklist to the  M3U8 varible below. 

Select the feed that you want to extract frames from by assigning the **M3U8** variable.


In [ ]:
M3U8 = "http://wzmedia.dot.ca.gov/D4/N1_at_Presidio_Tunnel.stream/chunklist_w665039362.m3u8"
M3U8 = "http://wzmedia.dot.ca.gov/D4/W92_at_El_Camino_Real.stream/chunklist_w2118793021.m3u8"
M3U8 = "http://wzmedia.dot.ca.gov/D4/S101_at_17th_St.stream/chunklist_w1949748683.m3u8"
M3U8 = "https://56cdb389b57ec.streamlock.net:1935/live/hbnorth.stream/chunklist_w790486524.m3u8"
M3U8 = "https://cdn-004.whatsupcams.com/hls/hr_zadar4.m3u8"
#M3U8 = "https://cdn-001.whatsupcams.com/hls/hr_zadar3.m3u8"
M3U8 = "https://cdn-001.whatsupcams.com/hls/it_venezia01.m3u8"


cams_chunk_url = M3U8
cams_base = video.extract_base(cams_chunk_url)

## Collect a set of sample frames

Collect some frames to be used for defining the crop region.

In [ ]:
def collect_frames(chunk_url, count=5):
    """collects fr ms making 'count' requests for chunks
       args:
            count : the number of chunks to fetc
        return:
            list of frames
    """
    cams_base = video.extract_base(chunk_url)
    collected_segment = collections.deque(maxlen=10)
    crop_frames = list()
    
    for idx in range(count):
        cams_response = requests.get(chunk_url)
        cams_chunklist = cams_response.text
        segments = video.m3u8_segments(cams_chunklist, cams_base)
    
        for segment in segments:
            # only get the first frame
            if segment not in collected_segment:
                frame = video.display_frame(segment)
                crop_frames.append(frame[0])
                print("segment:", segment)
                collected_segment.append(segment)
            else:
                #print("skip segment:", segment)
                pass
    return crop_frames


In [ ]:
CROP_FRAMES = collect_frames(M3U8, 3)
print("\n\nCollected {} frames".format(len(CROP_FRAMES)))

## Select a frame to define the crop region

#### Use  'crop'  button to select the frame

In [ ]:
def frame_selected(idx):
    global FRAME_IDX
    print("Use frame {} to work out cropping region.".format(idx))
    FRAME_IDX = idx
    
imgsel = video.image_select(CROP_FRAMES)
imgsel.ignition(frame_selected)


## Define crop region using the selected image.
#### Use  'Save Crop Sizes' when you're satisfied with the crop region.

In [ ]:
SHAPE = None
def grabCropShape(image_name, shape):
    global SHAPE
    SHAPE = shape
    print("set SHAPE ", image_name, shape, flush=True)

try :
    print("Using frame #{} to specify crop region.".format(FRAME_IDX))
except NameError as e:
    print("\n\nERROR * You must select the 'crop' button above to commit your selection.\n\n")


crop([CROP_FRAMES[FRAME_IDX]], callback=grabCropShape)


## Verify the cropped region to on the collected frames 

In [ ]:
 
if SHAPE is None:
    print("\n\nYou must select the 'Save Crop Sizes' button above to commit crop region.\n\n")
else:
    print("Using shape : {}".format(SHAPE))
    video.collected_frames(CROP_FRAMES, SHAPE)

### dashboard

Collect frames from the video feed, crop and display.

Threaded to get the button pushes. 

In [ ]:

class stream_cropped_video():

    def stop_clicked(self, b):
        self.stream_action.value = "initiate STOP"
        self.stop = True
    
    def transmit_clicked(self, b):
        if self.transmit:
            self.transmit = False
            self.stream_action.value = "transmit stopping"
            self.transmit_button.description = "Transmit * Start"
        else:
            self.transmit = True
            self.stream_action.value = "transmit countinue : {}".format(self.transmit_count)
            self.transmit_button.description = "Transmit * Stop"


    
        
    def __init__(self, cams_chunk_url=None, shape=None, wait_time=1, frame_modulo=24, kafka_producer=None, kafka_topic=None):
        """dashboard to send frames to streams
           args
                cams_chunk_url : M3U8 url, contents should have .ts files
                shape : value returned from cropping tool, region of image to be cropped
                
                wait_time : seconds to wait between 
                frame_modulo : send ever # frame 
                
        """
        self.cams_chunk_url = cams_chunk_url
        self.cams_base = cams_base = video.extract_base(cams_chunk_url)
        self.shape = shape
        self.wait_time = wait_time
        self.frame_modulo = frame_modulo
        self.kafka_prod = kafka_producer
        self.kafka_topic = kafka_topic
        self.active_segment = None
        
        self.stream_action = widgets.Label(layout={'border': '1px solid green','width':'61%'})
        self.stream_widget = widgets.Output(layout={'border': '1px solid red','width':'61%','height':'270pt'})
        self.stream_status = widgets.Label(layout={'border': '1px solid green','width':'61%'})
        self.transmit_button = widgets.Button(description="Transmit * Start ",layout={'width':'30%'})
        self.stop_button = widgets.Button(description="Stop Collecting frames",layout={'width':'30%'})
        self.stop = False
        self.transmit = False
        self.transmit_count = 0
        
        self.button_bar = widgets.HBox([self.transmit_button, self.stop_button])
        display(VBox([self.stream_action, self.stream_widget, self.button_bar, self.stream_status]))
        
        self.stop_button.on_click(self.stop_clicked)
        self.transmit_button.on_click(self.transmit_clicked)
        
        self.displayed_segment = collections.deque(maxlen=10)
        
        self.execute = threading.Event()
        self.thread = None


      
        
    def frame_crop_display(self, frame):
        """crop frame and display
        
        Returns:  encoded image that can be sent to streams
        Notes: 
             * crop region is shape
        """
        cropped_encoded_image = frame
        if self.shape is not None:
            cropped_encoded_image = video.crop_frame(frame, self.shape)
        with self.stream_widget:
            display(video.decode_img(cropped_encoded_image))
            clear_output(wait=True)
        return cropped_encoded_image
            
    def fetch_frames(self, segment, frame_modulo=24, process_frame=None):
        """fetch frames and process them 
        Args: 
            segment['segment] : url (*.ts) of video segment
            process_frame : function that processes the fetched frame, 
                            this.frame_crop_display(), stream.show_frame()
        Notes:
            * process_thread is interesting but maybe too 'cute'
    """
        cap = cv2.VideoCapture(segment['segment'])
        count = 0
        frames = list()
        image_encoded_frames = list()
        if (cap.isOpened()):
            while True:
                ret, frame = cap.read()
                if ret is False:
                    if self.stream_status is not None: self.stream_status.value = "ENDFrame @ {}".format(count)
                    return frames, image_encoded_frames
                if count % frame_modulo == 0:
                    frames.append(frame)
                    image_encoded_frames.append(process_frame(frame))

                count += 1
        else:
            print("Open failed....")
        return frames,image_encoded_frames
    
    def send_encoded_image(self, image_encoded, CHUNK_SIZE=100000):
        """send one encoded frame via kafka
        Args:
            image_encoded : image to send
            CHUNK_SIZE : maximum size to send
        """
        video_url = self.active_segment['segment']
        chunks = [image_encoded[i * CHUNK_SIZE:(i + 1) * CHUNK_SIZE] for i in
                  range((len(image_encoded) + CHUNK_SIZE - 1) // CHUNK_SIZE)]
        # send the chunks.
        for idx, chunk in enumerate(chunks):
            logging.debug("chunking - {}  #chunks :{} idx:{} len(chunk):{}".format(video_url, len(chunks), idx, len(chunk)))
            chunk_content = {'video': video_url,
                   'frame': self.transmit_count,
                   'chunk_idx':idx,
                   'chunk_total':len(chunks),
                   'timestamp': datetime.datetime.utcnow().isoformat() + 'Z',
                   'data': chunk
            }
            self.kafka_prod.send(self.kafka_topic, value=json.dumps(chunk_content).encode('utf-8'))
        ## finish the frame frame
        chunk_complete = {'video': video_url,
               'frame': self.transmit_count,
                'chunk_idx': len(chunks),
                'chunk_total': len(chunks),
               'timestamp': datetime.datetime.utcnow().isoformat() + 'Z',
               'data': ""
               }
        logging.info("Transmit frame #{}".format(chunk_content["frame"]))
        self.kafka_prod.send(self.kafka_topic, value=json.dumps(chunk_complete).encode('utf-8'))


    
    
    def kafka_frames(self, image_encoded_frames):
        if self.transmit:
            self.stream_action.value = "send"
            self.transmit_count += len(image_encoded_frames)
            self.stream_status.value = "transmit count : {}".format(self.transmit_count)
            for image_encoded in image_encoded_frames:
                self.send_encoded_image(image_encoded)

    
    def fetchFrames_thread(self):
        self.stop_button.disabled = True
        stop_action = lambda w: self.ignition(False)
        self.stop_button.on_click(stop_action)
        self.stop_button.disabled = False
        self.transmit = False
        self.execute.set()
        
        while self.execute.is_set():
            cams_response = requests.get(self.cams_chunk_url)
            cams_chunklist = cams_response.text
            segments = video.m3u8_segments(cams_chunklist, self.cams_base)
            time.sleep(self.wait_time)
            for segment in segments:
                self.active_segment = segment
                if self.stop is True: break
                if segment not in self.displayed_segment:
                    self.stream_action.value = "{} secs of frames in : {}".format(segment['secs'], segment['segment'])
                    frames,image_encoded_frames = self.fetch_frames(segment, process_frame=self.frame_crop_display, frame_modulo=self.frame_modulo)
                    self.stream_status.value = "Fetched ... {} frames.".format(len(frames))
                    self.kafka_frames(image_encoded_frames)
                    self.displayed_segment.append(segment)
                else:
                    self.stream_status.value = "skip segment:{}".format(segment)
         
        self.stop_button.description = "Stopped"

    def ignition(self, start):
        if start:
            self.thread = threading.Thread(target=self.fetchFrames_thread, name="RenderFrames" )
            self.thread.start()
        else:
            self.execute.clear()
            self.stop_button.description = "Stopping"
            self.stop_button.disabled = True
            self.transmit_button.disabled = True

creds = json.loads(credential.magsEventStream)
producer = video.kafka_producer(creds,)

scv = stream_cropped_video(M3U8, SHAPE, 
                           wait_time=2, 
                           frame_modulo=12, 
                           kafka_producer=producer, 
                           kafka_topic='VideoFrame')
scv.ignition(True)
print("done")
